## 1. Libraries and Packages

In [1]:
import requests
import lxml.html as lh
import pandas as pd
#pip install requests beautifulsoup
#!pip install selenium
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import json
# datetime library
import datetime
from datetime import datetime, timedelta

import re
import smtplib
import imghdr
import cgi
import ssl
import html
import uuid
import os
from email.mime.multipart import MIMEMultipart
from email.mime.text      import MIMEText
from email.mime.image     import MIMEImage
from email.header         import Header


## 2. Webscrapping data ##

### 2.1 Selenium Tutorial ###

In [2]:
# QUICK SELENIUM (webscraping robot) INSTALLATION GUIDE

# Step 1: Download chromedriver and create and extract to file path
# First need to download chromedriver (https://chromedriver.chromium.org/) to allow selenium to be used
# Create folder C:\Drivers\Chrome and extract the downloaded chromedriver file to this location

# select chromedriver path from directory 
driver =webdriver.Chrome(executable_path="C:\Drivers\Chrome\chromedriver.exe")


# Step 2: Follow guide below to add to system environment, stop before Steps to run Selenium Tests on Chrome Browser
# LINK: https://www.browserstack.com/guide/run-selenium-tests-using-selenium-chromedriver )




In [3]:
# FOLLOW GUIDE to learn how to find XPaths https://www.youtube.com/watch?v=b5pCd6-ZvQ0&t=317s&ab_channel=SDET-QAAutomationTechie 

### 2.2 Webscrapping IESO forecasted data and current yearly peaks ###
https://www.ieso.ca/peaktracker

In [4]:
# Direct driver to the website that will be scraped
driver.get("https://www.ieso.ca/peaktracker")
# obtaining daily energy consumption info from IESO, based on IESO predictive algos
# this is a really weird table format, the 2nd div corresponds to the column and the tr xpath tag counts up from 1 to 24, so leaving both blank 
# (no [1][3] beside them, find_elements will count them all)
all_hours=len(driver.find_elements_by_xpath("//*[@id='demandForecast']/div[2]/div/table/tbody/tr/td[1]"))
all_values = len(driver.find_elements_by_xpath("//*[@id='demandForecast']/div[2]/div/table/tbody/tr/td[2]"))

# obtaining current year top 5 peak information
rows_h=len(driver.find_elements_by_xpath("//*[@id='ieso-page-well']/div[7]/div/div[1]/div/div[3]/table/tbody/tr"))
cols_h= len(driver.find_elements_by_xpath("//*[@id='ieso-page-well']/div[7]/div/div[1]/div/div[3]/table/tbody/tr[1]/td"))

In [5]:
# view number of rows and cols and compare with website to ensure correct number
print("total individual hours:")
print(all_hours)

print("total individual value:")
print(all_values)
print("Historical Peak Rows:")
print(rows_h)
print('Historical peak columns:')
print(cols_h)

total individual hours:
24
total individual value:
24
Historical Peak Rows:
10
Historical peak columns:
6


In [6]:
# create blank dataframe of series (1 column) of data scraped weekly forecast data, later transformed into table
scraped_df = pd.DataFrame()
# create blank dataframe of series (1 column) of data scraped historical peak data, later transformed into table
scraped_df_h = pd.DataFrame()

In [7]:
# creating for loop for each row 
#Scraping data from forecast table and saving to df scraped_df

for t in range (1,all_values+1):
    hour=driver.find_element_by_xpath("//*[@id='demandForecast']/div[2]/div/table/tbody/tr["+str(t)+"]/td[1]").text
    value=driver.find_element_by_xpath("//*[@id='demandForecast']/div[2]/div/table/tbody/tr["+str(t)+"]/td[2]").text
    # appending to df...
    scraped_df = scraped_df.append({'hour': hour, 'Maximum_Forecasted_Ontario_Demand_(MW)':value},ignore_index=True)
    
    
#Scraping data from historical peaks table and saving to df scraped_df_h       
for r in range (1,rows_h+1):
    # Path for driver to take value from
    Day=driver.find_element_by_xpath("//*[@id='ieso-page-well']/div[7]/div/div[1]/div/div[3]/table/tbody/tr["+str(r)+"]/td[2]/span").text
    Hour=driver.find_element_by_xpath("//*[@id='ieso-page-well']/div[7]/div/div[1]/div/div[3]/table/tbody/tr["+str(r)+"]/td[3]/span").text
    Ontario_Demand=driver.find_element_by_xpath("//*[@id='ieso-page-well']/div[7]/div/div[1]/div/div[3]/table/tbody/tr["+str(r)+"]/td[4]/span").text
        
    # appending to df...
    scraped_df_h = scraped_df_h.append({'Day': Day,'Hour': Hour, 'Ontario_Demand_(MW)':Ontario_Demand},ignore_index=True)

### 2.3 Viewing dataframes (smart tables we made) ###

In [8]:
# print table to check if forecast scrape was successful
print(scraped_df)

   Maximum_Forecasted_Ontario_Demand_(MW) hour
0                                  20,478   14
1                                  20,764   15
2                                  21,267   16
3                                  21,717   17
4                                  21,771   18
5                                  21,576   19
6                                  21,050   20
7                                  20,485   21
8                                  19,224   22
9                                  17,652   23
10                                 16,251   24
11                                 15,143    1
12                                 14,381    2
13                                 13,904    3
14                                 13,762    4
15                                 14,426    5
16                                 15,366    6
17                                 16,383    7
18                                 17,241    8
19                                 17,988    9
20           

In [9]:
# print table to check if historical peak scrape was successful
print(scraped_df_h)

             Day Hour Ontario_Demand_(MW)
0  June 28, 2021   18              22,258
1  June 29, 2021   12              21,711
2  July 06, 2021   17              21,655
3  July 14, 2021   17              21,368
4  June 07, 2021   17              21,340
5  June 09, 2021   17              21,157
6  July 15, 2021   17              20,971
7  July 05, 2021   17              20,952
8  June 27, 2021   18              20,904
9  July 13, 2021   17              20,748


## 3. Data Cleaning ##

In [10]:
#check to see if dtypes are as desired 
scraped_df.dtypes

Maximum_Forecasted_Ontario_Demand_(MW)    object
hour                                      object
dtype: object

In [11]:
#check to see if dtypes are as desired 
scraped_df_h.dtypes

Day                    object
Hour                   object
Ontario_Demand_(MW)    object
dtype: object

In [12]:
# checking position of hour in dataframe (row, column)
print(scraped_df.iloc[0,1])

14


In [13]:
# checking position of predicted energy consumption in dataframe (row, column)
scraped_df.iloc[0,0]

'20,478'

In [14]:

# replacing all "," in order to be able to convert from object string to int

for i in range (0,all_hours):
    
    scraped_df.iloc[i,0] = re.sub(r'\,', ' ', scraped_df.iloc[i,0]) # remove ","
    scraped_df.iloc[i,0] = re.sub(' +',' ',scraped_df.iloc[i,0]) # remove more than one space 
    scraped_df.iloc[i,0] = re.sub(r'\s+','',scraped_df.iloc[i,0])

for i in range (0,rows_h):
    
    scraped_df_h.iloc[i,2] = re.sub(r'\,', ' ', scraped_df_h.iloc[i,2]) # remove ","
    scraped_df_h.iloc[i,2] = re.sub(' +',' ',scraped_df_h.iloc[i,2]) # remove more than one space 
    scraped_df_h.iloc[i,2] = re.sub(r'\s+','',scraped_df_h.iloc[i,2])
    
    

In [15]:
## above code isnt working so added extra conversion, for some reason this one works on Object type    
scraped_df['Maximum_Forecasted_Ontario_Demand_(MW)']=scraped_df['Maximum_Forecasted_Ontario_Demand_(MW)'].astype(int)

scraped_df_h['Ontario_Demand_(MW)']=scraped_df_h['Ontario_Demand_(MW)'].astype(int)

In [16]:
# check to see if commas were removed
print(scraped_df_h.iloc[4,2])

21340


## 4. Data Analysis ##

### 4.1 Peak hour of the day and 5th highest yearly peak ###

In [17]:
df = pd.DataFrame()

## obtaining the max forecasted energy use value for the day (which the IESO would base the peak hour on)

df=scraped_df[scraped_df['Maximum_Forecasted_Ontario_Demand_(MW)']==scraped_df['Maximum_Forecasted_Ontario_Demand_(MW)'].max()]
df=df.reset_index(drop=True)
daily_peak = df.iloc[0,0]

## obtaining the value for the 5th highest peak of the year , used later to determine if the forecasted peak is of concern
fifth_peak = scraped_df_h.iloc[4,2]

In [18]:
daily_peak

21771

### 4.2 Datetime formatting ###
Formatting peak hour into proper datetime format ###

In [19]:
# get current date from date time in format 
today = pd.to_datetime("today").strftime("%m/%d/%Y")

# formatting for datetime (need to match to proper format of other python file used later) 
df['hour']=df['hour'].astype(int)
# hour ending -> hour starting
hour_converter = df.iloc[0,1]-1
datetime_converter = ":00:00"
blanks = " "
        
# creating peak_date variable for textfile
potential_peak_date = today+blanks+str(hour_converter)+datetime_converter
potential_peak_date

'07/19/2021 17:00:00'

### 4.3 Risk Classfication ###
Classifying image to use based on severity of daily peak

In [20]:
# classifying risk level based on the severity of the daily peak
# there is an assosciated image with every bin that will be embedded into the email later in code
risk_level = 'Default'
if daily_peak < 20000:
    risk_level='Very Low'
    path = u'C:/Users/ty11.LAPTOP-TSRNCK40/OneDrive/Documents/Rankin Co/0.3x/20000.png'
if daily_peak >= 20000 and daily_peak < 20500:
    risk_level='Moderate'
    path = u'C:/Users/ty11.LAPTOP-TSRNCK40/OneDrive/Documents/Rankin Co/0.3x/20500.png' 
if daily_peak >= 20500 and daily_peak < 21000:
    risk_level='Moderate to High'
    path = u'C:/Users/ty11.LAPTOP-TSRNCK40/OneDrive/Documents/Rankin Co/0.3x/21000.png'
if daily_peak >= 21000 and daily_peak < 21750:
    risk_level='High'
    path = u'C:/Users/ty11.LAPTOP-TSRNCK40/OneDrive/Documents/Rankin Co/0.3x/21500.png'
if daily_peak >= 21750:
    risk_level='Extremely High'
    path = u'C:/Users/ty11.LAPTOP-TSRNCK40/OneDrive/Documents/Rankin Co/0.3x/22000.png'

In [21]:
# view risk level
risk_level

'Extremely High'

## 5. Alert Email ##

### 5.1 Email Construction ###
Construct message for email, will only be sent if conditions are met:

Condition 1: Daily peak > 5th highest peak of this year

Condition 2: Daily peak > 20500MW


In [25]:
# creating email body and subject line and storing in string variable
message="\n\n There is {} risk of top 5 peak energy hour anticipated today,  {}, between {} and {} EST, with a forecasted Ontario demand of: {} MW. \n\n Have a great day!".format(risk_level,today,str(hour_converter)+datetime_converter,str(hour_converter+1)+datetime_converter,daily_peak)
subject_line = 'Peak Alert System: ' + risk_level + ' risk of peak from ' + str(hour_converter) + datetime_converter + ' to ' +str(hour_converter+1) + datetime_converter + ' EST'

### 5.2 Embedding  image in an email ###
Fucking hard:
https://stackoverflow.com/questions/19171742/send-e-mail-to-gmail-with-inline-image-using-python
https://code.activestate.com/recipes/473810-send-an-html-email-with-embedded-image-and-plain-t/


In [23]:
## follow those guides and best of luck if this breaks , worst case
## remove images and just send message variable made above
img = dict(title=u'Picture report…', path=path, cid=str(uuid.uuid4()))


msg = MIMEMultipart('related')
msg['Subject'] = Header(u'{}'.format(subject_line), 'utf-8')
msg['From'] = "rankin.alerts@gmail.com"
msg['To'] = "rankin.alerts@gmail.com"
msg_alternative = MIMEMultipart('alternative')
msg.attach(msg_alternative)


msg_text = MIMEText(u'[image: {title}]'.format(**img), 'plain', 'utf-8')

msg_alternative.attach(msg_text)

msg_html = MIMEText(u' {} <div dir="ltr">'
                     '<img src="cid:{cid}" alt="{alt}"><br></div>'
                    .format(message, alt=html.escape(img['title'], quote=True), **img),
                    'html', 'utf-8')

msg_alternative.attach(msg_html)

with open(img['path'], 'rb') as file:
    msg_image = MIMEImage(file.read(), name=os.path.basename(img['path']))
    msg.attach(msg_image)
msg_image.add_header('Content-ID', '<{}>'.format(img['cid']))



### 5.3 Sending email  ###
Check to see if conditions are met, if so send the alert, if not the program ends until the following day

In [24]:
## open text file to append string to
file_object = open('C:/Users/ty11.LAPTOP-TSRNCK40/OneDrive/Documents/predicted_peaks.txt', 'a')

# check to see conditions are met to warrant potential peak
if daily_peak > 20500 and daily_peak>fifth_peak:
        #*********************
        #creating potential_peak variable to store peaks and transfer to text file
    potential_peak= daily_peak
    potential_peak_date = potential_peak_date
    
#############################SEND EMAIL##################################################
    Sender_Email = "rankin.alerts@gmail.com"
    
    Password = '@Penguins12'

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
        smtp.login(Sender_Email, Password)              
        smtp.sendmail(msg['From'],msg['To'],msg.as_string())


########################################################################################
        
# Append potential peaks date and value at the end of file (1 entry per line)
    file_object.write(str(potential_peak_date))
    file_object.write(',')
    file_object.write(str(potential_peak))
    file_object.write('\n')  
    
else:
    print("No peaks today")
        
# Close the textfile
file_object.close()